# 9. Property based testing and data integrity testing

## Property based testing with Hypothesis

So far all the tests we have written/write are done at a basic level in which you take 1 test and write test cases for which ti should pass or fail.  But what about edge cases? How do you know when you have actually found a bug and not an edge case for your model/algorithm or else?

The [Hypothesis package](https://hypothesis.readthedocs.org/en/latest/) is very useful in these situtations as it performs property based testing. So instead of taking let's say 1 or 2 test cases for a single function hypothesis tests a whole range of parameters for each test case. 

Not clear? Ok imagine we do not know about the existence of the `modullo` operator so we have instead writen a function that tests divisibility by 11 based on the following property: 
> A number is divisible by 11 if and only if the alternating (in sign) sum of the number’s digits is 0

Let's create a `main.py` script with our function:

In [1]:
def divisible_by_11(number):
    """Uses above criterion to check if number is divisible by 11"""
    string_number = str(number)
    alternating_sum = sum([(-1) ** i * int(d) for i, d
                           in enumerate(string_number)])
    return alternating_sum == 0

And we can demonstrate that this works by running some examples

In [ ]:
for k in range(10):
    print(divisible_by_11(11 * k))

Now let's write a test case for the above example! And we are going to save this in `test_main.py`

In [21]:
import pytest


def test_divisible_by_11():
    for k in range(10):
        assert divisible_by_11(11*k)
        assert divisible_by_11(121)
        assert divisible_by_11(12122)

In [11]:
test_divisible_by_11()

The tests passed so no errors were raised at all. The above tests the first 10 numbers divisible by 11 and also some specific tests (121 and 12122).


At this point we could be very happy and proud of ourselves: we have tested well written software that can be shipped and used by researchers world wide to test the divisibility of a number by 11.


That is how mathematics breaks... so let's use hypothesis to demonstrate. Let's create a `test_property_main.py` script:

In [22]:
from hypothesis import given  # define the inputs
import hypothesis.strategies as st

@given(k=st.integers(min_value=1)) # main decorator
def test_divisible_by_11(k):
    assert divisible_by_11(11*k)

And we can run the test:
```
$  python -m pytest test_main.py
============================= test session starts ================
=============
platform win32 -- Python 3.6.5, pytest-3.5.1, py-1.5.3, pluggy-0.6
.0
plugins: nbval-0.9.0, hypothesis-3.58.1
collecting 0 items
collecting 1 item
collected 1 item


test_property_main.py F
       [100%]

================================== FAILURES ======================
=============
____________________________ test_divisible_by_11 ________________
_____________

    @given(k=st.integers(min_value=1)) # main decorator
>   def test_divisible_by_11(k):

test_main.py:8:
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

...

test_main.py:9: AssertionError
--------------------------------- Hypothesis ---------------------
-------------
Falsifying example: test_divisible_by_11(k=28)
========================== 1 failed in 0.60 seconds ==============
=============


```
We get an error. Right at the top we get the `Falsifying example` so we see that our function fails for `k=28`.  The number resulting from $11x28$ is not divisible by 11 by construction. 

At this point we can think of the following correct definition for a number to be divisuble by 11:

> A number is divisible by 11 if and only if the alternating (in sign) sum of the number’s digits is divisible by 11.

So we can now modify `main.py` :

In [23]:
def divisible_by_11(number):
    """Uses above criterion to check if number is divisible by 11"""
    string_number = str(number)
    # Using abs as the order of the alternating sum doesn't matter.
    alternating_sum = abs(sum([(-1) ** i * int(d) for i, d
                               in enumerate(string_number)]))
    # Recursively calling the function
    return (alternating_sum in [0, 11]) or divisible_by_11(alternating_sum)

Running the tests:

```
$ python -m pytest test_property_main
.
----------------------------------------------------------------------
Ran 1 test in 0.043s

OK
```